# Imports

In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

# Reading Text

In [2]:
def read_text(data):
    article = data.split(". ")
    sentences = []

    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

# Recording Sentence Similarity

In [3]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

# Build Similarity Matrix

In [4]:
def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: 
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

# Generating Final Summary

In [5]:
def generate_summary(data, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    sentences =  read_text(data)

    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))
    
    summarize_text[len(summarize_text) - 1] += '.'
    
    return '. '.join(summarize_text)

In [6]:
val = '''SALT LAKE CITY — The State of Utah now has a better timeline on when the first group of Afghan refugees could arrive in the Beehive State. The Department of Workforce Services Refugees Services Office said Friday that refugees could land in Salt Lake City as early as the second week of October.

The state, along with multiple agencies and local organizations have been busy behind the scenes trying to coordinate resources, from employment to financial aid, to healthcare access, to goods and housing.

READ: More than 750 Afghan refugees expected to resettle in Utah over the coming months

"We all saw the videos in Afghanistan of people rushing to get to the airport and trying to exit and leave," said State of Utah Refugee Services Office Director Asha Parekh.

Parekh said some arriving are humanitarian parolees, who will not have access to federal funds like those with special immigrant visas. That's because the emergency situation in Afghanistan, she said, meant many left their homes in an urgent rush without time to fill out all the paperwork or to complete their legal status.

She said many individuals who made it to the US are at various military bases, waiting to go through medical screening and get work authorization.

WATCH: After fleeing Kabul, Utah restaurant owner finds success sharing Afghan culture with community

Meanwhile, offices like Parekh's are busy planning for as many as 750 refugees to arrive in Salt Lake City.

"The Governor's Advisory Board has set up three working groups," she said. "One is to address the issue of housing because we know that there is limited housing in our state."

Parekh explained a task force is focusing on securing housing, in what's already a tight market.

"We're not sure yet how that process will work," she said. "We hope that it's smooth and clean, but recognizing that people, that Afghan humanitarian parolees, have experienced trauma."

Wanting to help ease that tough transition, hundreds of landlords stand ready to assist.

Paul Smith, Executive Director of the Utah Apartment Association said they sent out an email blast to all of their landlords. The response was overwhelming.

"The owners of over 20,000 units have already volunteered to work with refugees," he said.

Smith said they're still working out exact logisitics, but they're hoping to use COVID-19 state aid to pay six months of rent for incoming refugees.

"The State Department said they'll give us about a week's worth notice," he described. "We'll reach out to our list of landlords and say, 'Do you have two-bedroom in Sugar House available?' Or, 'Do you have a three-bedroom in Utah County?' And we'll identify a unit, hopefully have it ready by the time the refugees get here."

In addition to housing, there are other groups coordinating specifics to help furnish apartments and find people jobs. Parekh explained that healthcare partners like Intermountain Healthcare will help with medical assistance.

She knows how important it'll be to have these resources on standby, for as smooth a resettlement process as possible.

"Being able to help them move into an apartment that they can call their own in this new land, is going to make a huge difference to their experience," she said.'''

In [7]:
a = generate_summary(val, 3)
a

'The Department of Workforce Services Refugees Services Office said Friday that refugees could land in Salt Lake City as early as the second week of October.\n\nThe state, along with multiple agencies and local organizations have been busy behind the scenes trying to coordinate resources, from employment to financial aid, to healthcare access, to goods and housing.\n\nREAD: More than 750 Afghan refugees expected to resettle in Utah over the coming months\n\n"We all saw the videos in Afghanistan of people rushing to get to the airport and trying to exit and leave," said State of Utah Refugee Services Office Director Asha Parekh.\n\nParekh said some arriving are humanitarian parolees, who will not have access to federal funds like those with special immigrant visas. SALT LAKE CITY — The State of Utah now has a better timeline on when the first group of Afghan refugees could arrive in the Beehive State. That\'s because the emergency situation in Afghanistan, she said, meant many left thei